<a href="https://colab.research.google.com/github/PaulaBergamasco/MetoriaBCI/blob/main/Curacion_de_DataFrames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pandas._libs.tslibs.timestamps import Timestamp
from datetime import datetime
import pandas as pd

import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
files_names = ['/content/drive/MyDrive/Mentoria /AA 0 - 2020-10-28.csv' , '/content/drive/MyDrive/Mentoria /AA 1 - 2020-11-24.txt', '/content/drive/MyDrive/Mentoria /AA 2 - 2020-11-24.txt',
               '/content/drive/MyDrive/Mentoria /HA 1 - 2020-11-24.txt', '/content/drive/MyDrive/Mentoria /JA 1 - 2020 - 11-19.txt', '/content/drive/MyDrive/Mentoria /JA 2 - 2020-11-24.txt',
               '/content/drive/MyDrive/Mentoria /MA 1 - 2020-12-18.txt']

In [ ]:
columns = ['SampleIndex', 'Canal1', 'Canal2', 'Canal3', 'Canal4', '5', '6', '7', 'Stimulus', 'Time', 'TimeStamp']

In [ ]:
AA0 = pd.read_csv(files_names[0], names=columns)
AA1 = pd.read_csv(files_names[1], names=columns, skiprows=10)
AA2 = pd.read_csv(files_names[2], names=columns, skiprows=10)
HA1 = pd.read_csv(files_names[3], names=columns, skiprows=10)
JA1 = pd.read_csv(files_names[4], names=columns, skiprows=10)
JA2 = pd.read_csv(files_names[5], names=columns, skiprows=10)
MA1 = pd.read_csv(files_names[6], names=columns, skiprows=10)
MA1.head(3)

,SampleIndex,Canal1,Canal2,Canal3,Canal4,5,6,7,Stimulus,Time,TimeStamp
0,0,717.21,-9.85,1325.26,1241.21,0.0,0.0,0.0,99,10:18:14.775,1608297494775
1,1,720.58,-6.43,1327.61,1239.30,0.0,0.0,0.0,99,10:18:14.790,1608297494790
2,2,731.72,16.29,1335.43,1253.36,0.0,0.0,0.0,99,10:18:14.790,1608297494790


In [ ]:
"""
Canal1 EEG data en microvoltios
Canal2 EEG data en microvoltios
Canal3 EEG data en microvoltios
Canal4 EEG data en microvoltios
5 Accel Data sin datos (columna sin informacion util)
6 Accel Data sin datos (columna sin informacion util)
7 Accel Data sin datos (columna sin informacion util)
Stimulus Position Posicion del ojo durante la calibracion
                0 --> not looking; 1 --> looking left; 2 --> looking right, 99 --> NaN (default value)
Time Hora de la toma de muestra, no relevante
ya que vamos a crear nuestra linea de tiempo para los 200Hz de muestreo
TimeStamp Similar al anterior, provisto por OpenBCI no es relevante.
"""

'\nCanal1 EEG data en microvoltios\nCanal2 EEG data en microvoltios\nCanal3 EEG data en microvoltios\nCanal4 EEG data en microvoltios\n5 Accel Data sin datos (columna sin informacion util)\n6 Accel Data sin datos (columna sin informacion util)\n7 Accel Data sin datos (columna sin informacion util)\nStimulus Position Posicion del ojo durante la calibracion\n                0 --> not looking; 1 --> looking left; 2 --> looking right, 99 --> NaN (default value)\nTime Hora de la toma de muestra, no relevante\nya que vamos a crear nuestra linea de tiempo para los 200Hz de muestreo\nTimeStamp Similar al anterior, provisto por OpenBCI no es relevante.\n'

# Funciones para la curacion para las columnas no relevantes

In [ ]:
# Eliminar las columnas no relevantes
def eliminar_columnas(df):
    interesting_columns = ['SampleIndex','Canal1','Canal2','Canal3','Canal4', 'Stimulus']
    # Elimina las columnas que no son de interes
    for column in df:
        if column not in interesting_columns:
            df = df.drop(column, axis = 1)
    return df

In [ ]:
AA0 = eliminar_columnas(AA0)
AA1 = eliminar_columnas(AA1)
AA2 = eliminar_columnas(AA2)
HA1 = eliminar_columnas(HA1)
JA1 = eliminar_columnas(JA1)
JA2 = eliminar_columnas(JA2)
MA1 = eliminar_columnas(MA1)

In [ ]:
HA1.head()

,SampleIndex,Canal1,Canal2,Canal3,Canal4,Stimulus
0,0,1798.66,771.75,1753.60,1814.62,99
1,1,1730.72,706.07,1715.23,1763.01,99
2,2,1822.16,806.13,1764.67,1848.29,99
3,3,1848.34,840.87,1777.78,1861.13,99
4,4,1766.41,753.03,1722.41,1784.65,99


# Resta de media a los canales necesarios

In [ ]:
# Asumimos que si la media esta por encima de 300 hay ruido ya qye la señal de EEG normal
# oscila entre los -250 y 250 microvoltios
# La idea de este filtrado es eliminar esa componente de continua que seria ruido
def elimina_continua(df):
    canales = ['Canal1', 'Canal2', 'Canal3', 'Canal4']
    for canal in canales:
        if df[canal].mean() > 300:
            df[canal] = df[canal] - df[canal].mean()
    return df

In [ ]:
AA0 = elimina_continua(AA0)
AA1 = elimina_continua(AA1)
AA2 = elimina_continua(AA2)
HA1 = elimina_continua(HA1)
JA1 = elimina_continua(JA1)
JA2 = elimina_continua(JA2)
MA1 = elimina_continua(MA1)

# Eliminar los valores superiores a 250 e inferiores a -250

In [ ]:
HA1.describe().round()

,SampleIndex,Canal1,Canal2,Canal3,Canal4,Stimulus
count,137693.0,137693.0,137693.0,137693.0,137693.0,137693.0
mean,100.0,0.0,0.0,0.0,-0.0,71.0
std,58.0,117.0,127.0,113.0,127.0,44.0
min,0.0,-500.0,-588.0,-411.0,-436.0,1.0
25%,50.0,-89.0,-92.0,-93.0,-105.0,2.0
50%,100.0,3.0,12.0,12.0,12.0,99.0
75%,150.0,93.0,109.0,95.0,104.0,99.0
max,200.0,562.0,1036.0,469.0,358.0,99.0


In [ ]:
HA1[(HA1['Canal1'] < 251) & (HA1['Canal1'] > -251)].describe()

,SampleIndex,Canal1,Canal2,Canal3,Canal4,Stimulus
count,134363.000000,134363.000000,134363.000000,134363.000000,134363.000000,134363.00000
mean,100.010754,0.369360,2.183031,0.889423,1.243065,70.49210
std,58.052152,109.254034,121.634717,108.399288,121.476763,44.34044
min,0.000000,-250.985979,-587.525374,-372.490736,-384.456555,1.00000
25%,50.000000,-87.300979,-90.205374,-91.120736,-102.416555,2.00000
50%,100.000000,3.404021,12.094626,12.239264,12.263445,99.00000
75%,150.000000,91.834021,108.014626,94.629264,102.993445,99.00000
max,200.000000,250.964021,1035.714626,468.919264,332.913445,99.00000


In [ ]:
def filtro_rango(df):
    canales = ['Canal1', 'Canal2', 'Canal3', 'Canal4']
    for canal in canales:
        df = df[(df[canal] < 250) & (df[canal] > -250)]
    return df

In [ ]:
AA0 = filtro_rango(AA0)
AA1 = filtro_rango(AA1)
AA2 = filtro_rango(AA2)
HA1 = filtro_rango(HA1)
JA1 = filtro_rango(JA1)
JA2 = filtro_rango(JA2)
MA1 = filtro_rango(MA1)

In [ ]:
HA1.describe().round()

,SampleIndex,Canal1,Canal2,Canal3,Canal4,Stimulus
count,129852.0,129852.0,129852.0,129852.0,129852.0,129852.0
mean,100.0,3.0,7.0,3.0,4.0,70.0
std,58.0,104.0,112.0,103.0,115.0,44.0
min,0.0,-250.0,-250.0,-250.0,-250.0,1.0
25%,50.0,-83.0,-86.0,-87.0,-97.0,2.0
50%,100.0,6.0,16.0,15.0,15.0,99.0
75%,150.0,91.0,108.0,94.0,103.0,99.0
max,200.0,250.0,249.0,246.0,250.0,99.0


# Agregar Columna Fuente

In [ ]:
AA0['Fuente']= 'AA0'
AA1['Fuente']= 'AA1'
AA2['Fuente']= 'AA2'
HA1['Fuente']= 'HA1'
JA1['Fuente']= 'JA1'
JA2['Fuente']= 'JA2'
MA1['Fuente']= 'MA1'

# Agregar Linea Temporal de los 200Hz

In [ ]:
TimeAA0= pd.date_range(start ='2017-1-1', periods = len(AA0), freq='5ms')
TimeAA1= pd.date_range(start ='2017-1-1', periods = len(AA1), freq='5ms')
TimeAA2= pd.date_range(start ='2017-1-1', periods = len(AA2), freq='5ms')
TimeHA1= pd.date_range(start ='2017-1-1', periods = len(HA1), freq='5ms')
TimeJA1= pd.date_range(start ='2017-1-1', periods = len(JA1), freq='5ms')
TimeJA2= pd.date_range(start ='2017-1-1', periods = len(JA2), freq='5ms')
TimeMA1= pd.date_range(start ='2017-1-1', periods = len(MA1), freq='5ms')
AA0['STime']= TimeAA0.strftime('%M:%S.%f')
AA1['STime']= TimeAA1.strftime('%M:%S.%f')
AA2['STime']= TimeAA2.strftime('%M:%S.%f')
HA1['STime']= TimeHA1.strftime('%M:%S.%f')
JA1['STime']= TimeJA1.strftime('%M:%S.%f')
JA2['STime']= TimeJA2.strftime('%M:%S.%f')
MA1['STime']= TimeMA1.strftime('%M:%S.%f')

In [ ]:
AA0.head()

,SampleIndex,Canal1,Canal2,Canal3,Canal4,Stimulus,Fuente,STime
0,0,-1.86,-12.95,-9.18,-41.36,99,AA0,00:00.000000
1,1,10.77,3.47,6.87,-15.13,99,AA0,00:00.005000
2,2,87.61,65.61,88.78,32.23,99,AA0,00:00.010000
3,3,83.04,50.88,77.30,4.49,99,AA0,00:00.015000
4,4,8.07,-20.55,-6.68,-38.09,99,AA0,00:00.020000


In [ ]:
AA1.head()

,SampleIndex,Canal1,Canal2,Canal3,Canal4,Stimulus,Fuente,STime
0,0,188.245107,42.053325,144.616415,74.002857,99,AA1,00:00.000000
1,1,33.205107,-116.536675,-19.363585,10.962857,99,AA1,00:00.005000
3,3,43.925107,-103.906675,-35.243585,68.792857,99,AA1,00:00.010000
4,4,186.495107,28.013325,137.526415,80.272857,99,AA1,00:00.015000
5,5,38.685107,-138.666675,-28.633585,8.982857,99,AA1,00:00.020000


In [ ]:
AA2.head()

,SampleIndex,Canal1,Canal2,Canal3,Canal4,Stimulus,Fuente,STime
0,0,-106.521545,-213.002669,-201.22592,-143.46079,99,AA2,00:00.000000
1,1,-44.991545,-158.522669,-162.33592,-91.88079,99,AA2,00:00.005000
4,4,-89.991545,-206.832669,-186.63592,-128.52079,99,AA2,00:00.010000
5,5,-36.911545,-181.162669,-158.29592,-80.10079,99,AA2,00:00.015000
8,8,-97.401545,-212.832669,-196.98592,-122.06079,99,AA2,00:00.020000


In [ ]:
HA1.head()

,SampleIndex,Canal1,Canal2,Canal3,Canal4,Stimulus,Fuente,STime
0,0,-149.455979,-168.755374,-155.410736,-168.636555,99,HA1,00:00.000000
1,1,-217.395979,-234.435374,-193.780736,-220.246555,99,HA1,00:00.005000
2,2,-125.955979,-134.375374,-144.340736,-134.966555,99,HA1,00:00.010000
3,3,-99.775979,-99.635374,-131.230736,-122.126555,99,HA1,00:00.015000
4,4,-181.705979,-187.475374,-186.600736,-198.606555,99,HA1,00:00.020000


In [ ]:
JA1.head()

,SampleIndex,Canal1,Canal2,Canal3,Canal4,Stimulus,Fuente,STime
0,0,-114.51323,-117.521456,-90.507642,-92.035311,99,JA1,00:00.000000
1,1,-101.19323,-110.251456,-86.797642,-84.465311,99,JA1,00:00.005000
2,2,-93.88323,-105.481456,-86.627642,-83.395311,99,JA1,00:00.010000
3,3,-100.20323,-106.291456,-85.907642,-85.445311,99,JA1,00:00.015000
4,4,-101.65323,-104.951456,-83.377642,-83.505311,99,JA1,00:00.020000


In [ ]:
JA2.head()

,SampleIndex,Canal1,Canal2,Canal3,Canal4,Stimulus,Fuente,STime
216,15,223.150763,-120.7637,58.833857,107.253943,99,JA2,00:00.000000
223,22,237.960763,-63.1237,86.503857,187.713943,99,JA2,00:00.005000
224,23,194.250763,-130.4837,36.413857,98.303943,99,JA2,00:00.010000
227,26,216.610763,-66.2237,87.563857,181.233943,99,JA2,00:00.015000
228,27,182.440763,-128.6737,49.813857,98.233943,99,JA2,00:00.020000


In [ ]:
MA1.head()

,SampleIndex,Canal1,Canal2,Canal3,Canal4,Stimulus,Fuente,STime
0,0,-49.905006,-9.85,-23.809226,-52.29113,99,MA1,00:00.000000
1,1,-46.535006,-6.43,-21.459226,-54.20113,99,MA1,00:00.005000
2,2,-35.395006,16.29,-13.639226,-40.14113,99,MA1,00:00.010000
3,3,-42.395006,14.78,-10.369226,-42.74113,99,MA1,00:00.015000
4,4,-48.365006,0.03,-13.269226,-46.56113,99,MA1,00:00.020000


In [ ]:
df = pd.concat([AA0, AA1, AA2, HA1, JA1, JA2, MA1])

In [ ]:
df

,SampleIndex,Canal1,Canal2,Canal3,Canal4,Stimulus,Fuente,STime
0,0,-1.860000,-12.95,-9.180000,-41.36000,99,AA0,00:00.000000
1,1,10.770000,3.47,6.870000,-15.13000,99,AA0,00:00.005000
2,2,87.610000,65.61,88.780000,32.23000,99,AA0,00:00.010000
3,3,83.040000,50.88,77.300000,4.49000,99,AA0,00:00.015000
4,4,8.070000,-20.55,-6.680000,-38.09000,99,AA0,00:00.020000
...,...,...,...,...,...,...,...,...
69705,140,37.984994,36.59,21.540774,48.65887,99,MA1,05:48.215000
69706,141,41.934994,27.30,29.620774,46.61887,99,MA1,05:48.220000
69707,142,50.934994,27.28,35.680774,51.85887,99,MA1,05:48.225000
69708,143,43.424994,31.55,24.580774,40.89887,99,MA1,05:48.230000


# Curacion de datos que eliminan el valor 99 de stimulus

In [ ]:
df= df[df.Stimulus != 99]

In [ ]:
df.set_index(columns[0], inplace= True)

In [ ]:
df.to_csv(r'DataFrame_curado.csv', index = False)